## Cleaning

In [1]:
import pandas as pd
df = pd.read_csv("kolkata.csv", skiprows=9)

In [2]:
df.head()

,YEAR,DOY,T2M_MAX
0,2002,121,40.67
1,2002,122,39.94
2,2002,123,41.18
3,2002,124,40.96
4,2002,125,40.80


In [3]:
df.dropna(inplace=True)
df[df.isnull().any(axis=1)]
df['DATE'] = pd.to_datetime(df['YEAR'].astype(str) + df['DOY'].astype(str), format='%Y%j').dt.strftime('%d-%m-%Y')
df.drop(df.columns[[1]], axis=1)
df = df[['DATE', 'T2M_MAX', 'YEAR']]
df['HEATWAVE'] = df['T2M_MAX'] > 40
df['CITY'] = 'Kolkata'


In [4]:
df.head()

,DATE,T2M_MAX,YEAR,HEATWAVE,CITY
0,01-05-2002,40.67,2002,True,Kolkata
1,02-05-2002,39.94,2002,False,Kolkata
2,03-05-2002,41.18,2002,True,Kolkata
3,04-05-2002,40.96,2002,True,Kolkata
4,05-05-2002,40.80,2002,True,Kolkata


In [5]:
df.to_csv("cleaned_kolkata.csv", index=False)

## Population

In [6]:
import numpy as np
import pandas as pd
# 2001 and 2011 population
years = [2001, 2011]
population = [4572876, 4496694]

# interpolate for 2002 to 2010
interp_years = np.arange(2001, 2012)
interp_population = np.interp(interp_years, years, population)

# create dataframe 
df_pop = pd.DataFrame({
    'YEAR': interp_years,
    'POPULATION': interp_population.astype(int)
})
print(df_pop)

    YEAR  POPULATION
0   2001     4572876
1   2002     4565257
2   2003     4557639
3   2004     4550021
4   2005     4542403
5   2006     4534785
6   2007     4527166
7   2008     4519548
8   2009     4511930
9   2010     4504312
10  2011     4496694


In [7]:
# extrapoliate

In [8]:
growth_rate = ((4496694/4572876)**(1/10))-1

In [9]:
df_pop = df_pop[df_pop['YEAR'] < 2012].copy()

In [10]:
for year in range(2012, 2023):
    last_pop = df_pop.loc[df_pop['YEAR'] == year - 1, 'POPULATION'].values[0]
    new_pop = last_pop * (1 + growth_rate)
    df_pop.loc[len(df_pop)] = [year, int(new_pop)]
df_pop = df_pop.sort_values('YEAR').reset_index(drop=True)

In [11]:
print(df_pop.tail(15))

    YEAR  POPULATION
7   2008     4519548
8   2009     4511930
9   2010     4504312
10  2011     4496694
11  2012     4489145
12  2013     4481609
13  2014     4474086
14  2015     4466575
15  2016     4459077
16  2017     4451592
17  2018     4444119
18  2019     4436659
19  2020     4429211
20  2021     4421776
21  2022     4414353


In [12]:
df_pop.to_csv("population_kolkata.csv", index=False)

## Merging

In [13]:
import pandas as pd
df1 = pd.read_csv("cleaned_kolkata.csv")
df2 = pd.read_csv("population_kolkata.csv")
# merging
merged_df = pd.merge(df1, df2, on='YEAR', how='left')

In [14]:
merged_df.to_csv("merged_kolkata.csv", index=False)